In [ ]:
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

    output_path = 'detected_faces.jpg'
    cv2.imwrite(output_path, img)
    cv2.imshow('Detected Faces', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

detect_faces(r"C:\Users\jigya\OneDrive\Pictures\Camera Roll\OIP.jpeg")


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))

def extract_face_embedding(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    embedding = model.predict(img_array)
    return embedding

known_face_path = r"C:\Users\jigya\OneDrive\Pictures\Camera Roll\OIP.jpeg"  # Replace with the path to your known face image
known_face_embedding = extract_face_embedding(known_face_path)

np.save('known_face_embedding.npy', known_face_embedding)
print("Known face embedding saved.")


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg", input_shape=(224, 224, 3))

def extract_face_embedding(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    embedding = model.predict(img_array)
    return embedding
    
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def detect_faces(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face_path = 'detected_face.jpg'
        cv2.imwrite(face_path, face)
        
        face_embedding = extract_face_embedding(face_path)
        recognized = recognize_face(face_embedding)
        
        color = (0, 255, 0) if recognized else (0, 0, 255)
        label = "Recognized" if recognized else "Unknown"
        
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    output_path = 'recognized_faces.jpg'
    cv2.imwrite(output_path, img)
    cv2.imshow('Face Recognition', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def recognize_face(face_embedding, threshold=0.5):
    known_embedding = np.load('known_face_embedding.npy') 
    distance = np.linalg.norm(known_embedding - face_embedding)
    return distance < threshold

detect_faces(r"C:\Users\jigya\OneDrive\Pictures\Camera Roll\OIP.jpeg")
